In [2]:
# get NHS data

import zipfile
import urllib2
import os

source_url = 'ftp://ftp.nhtsa.dot.gov/GES/GES12/GES12_Flatfile.zip'
zip_name = 'GES12_Flatfile.zip'
cwd = os.getcwd()
dir_path = os.path.join(cwd, 'GES2012')
zip_path = os.path.join(dir_path, zip_name)

# We'll make a directory to play around with
# Then when done playing you can just delete the directory

if not os.path.exists(dir_path):
    os.makedirs(dir_path)
    
# Download the file from GES website if you haven't already
if not os.path.exists(zip_path):
    response = urllib2.urlopen(source_url)
    with open(zip_path, 'wb') as fh:
        x = response.read()
        fh.write(x)
        
# Extract all the files from that zipfile
with zipfile.ZipFile(os.path.join(dir_path, zip_name), 'r') as z:
    z.extractall(dir_path)

In [3]:
# see what we just unzipped
os.listdir(dir_path)

['2012GESFlatFileTXT.sas',
 'ACCIDENT.TXT',
 'CEVENT.TXT',
 'DAMAGE.TXT',
 'DISTRACT.TXT',
 'DRIMPAIR.TXT',
 'FACTOR.TXT',
 'GES12_Flatfile.zip',
 'MANEUVER.TXT',
 'NMCRASH.TXT',
 'NMIMPAIR.TXT',
 'NMPRIOR.TXT',
 'PARKWORK.TXT',
 'PERSON.TXT',
 'SAFETYEQ.TXT',
 'VEHICLE.TXT',
 'VEVENT.TXT',
 'VIOLATN.TXT',
 'VISION.TXT',
 'VSOE.TXT']

In [5]:
# Load data into python

import pandas as pd
import numpy as np
import sklearn

cwd = os.getcwd()
dir_path = os.path.join(cwd, 'GES2012')
input_file_path = os.path.join(dir_path, 'PERSON.TXT')

input_data = pd.read_csv(input_file_path, delimiter='\t')

sorted(input_data.columns)

['AGE',
 'AGE_IM',
 'AIR_BAG',
 'ALC_RES',
 'ALC_STATUS',
 'ATST_TYP',
 'BODY_TYP',
 'CASENUM',
 'DRINKING',
 'DRUGRES1',
 'DRUGRES2',
 'DRUGRES3',
 'DRUGS',
 'DRUGTST1',
 'DRUGTST2',
 'DRUGTST3',
 'DSTATUS',
 'EJECTION',
 'EJECT_IM',
 'EMER_USE',
 'FIRE_EXP',
 'HARM_EV',
 'HOSPITAL',
 'HOUR',
 'IMPACT1',
 'INJSEV_IM',
 'INJ_SEV',
 'LOCATION',
 'MAKE',
 'MAN_COLL',
 'MINUTE',
 'MOD_YEAR',
 'MONTH',
 'PERALCH_IM',
 'PER_NO',
 'PER_TYP',
 'PJ',
 'PSU',
 'PSUSTRAT',
 'P_SF1',
 'P_SF2',
 'P_SF3',
 'REGION',
 'REST_MIS',
 'REST_USE',
 'ROLLOVER',
 'SCH_BUS',
 'SEAT_IM',
 'SEAT_POS',
 'SEX',
 'SEX_IM',
 'SPEC_USE',
 'STRATUM',
 'STR_VEH',
 'TOW_VEH',
 'VEH_NO',
 'VE_FORMS',
 'WEIGHT']

In [6]:
# Clean up data
input_data.INJSEV_IM.value_counts()

0    100840
2     20758
1     19380
3      9738
5      1179
4      1178
6         4
dtype: int64

In [8]:
# only four cases where person died before crash, so just ignore.
# Some columns have missing values:

# Drop odd cases
input_data = input_data[input_data.INJSEV_IM != 6]

for column_name in input_data.columns:
    n_nans = input_data[column_name].isnull().sum()
    if n_nans > 0:
        print column_name, n_nans

MAKE 5162
BODY_TYP 5162
MOD_YEAR 5162
TOW_VEH 5162
SPEC_USE 5162
EMER_USE 5162
ROLLOVER 5162
IMPACT1 5162
FIRE_EXP 5162


In [13]:
print input_data.shape
data = input_data[~input_data.MAKE.isnull()]
discarded = data.pop('INJ_SEV')
target = data.pop('INJSEV_IM')
print data.shape


(153073, 58)
(147911, 56)


In [14]:
# One last preprocessing step we'll do it transform the response.
# Category 4 is a fatal injury, so we will encode as such

target = (target == 4).astype('float')

In [15]:
# Now we model!
# We will use ols, Ridge Regression, Lasso Regression,
# Gradient Boosting Machine, and a CART

from sklearn.linear_model import LinearRegression
from sklearn.metrics import roc_auc_score
from sklearn.grid_search import GridSearchCV

# Train on half of the data while reserving other half for
# model comparison
xtrain, xtest, ytrain, ytest = sklearn.cross_validation.train_test_split(
    data.values, target.values, train_size=0.5)

linreg = LinearRegression()
linreg.fit(xtrain, ytrain)

lr_preds = linreg.predict(xtest)
lr_perf = roc_auc_score(ytest, lr_preds)
print 'OLS: Area under the ROC curve = {}'.format(lr_perf)

OLS: Area under the ROC curve = 0.932776711135


In [16]:
from sklearn.linear_model import Ridge

ridge = GridSearchCV(Ridge(),
                    {'alpha': np.logspace(-10, 10, 100)})
ridge.fit(xtrain, ytrain)
ridge_preds = ridge.predict(xtest)
ridge_performance = roc_auc_score(ytest, ridge_preds)
print 'Ridge: Area under the ROC curve = {}'.format(ridge_performance)

Ridge: Area under the ROC curve = 0.932786784372


In [18]:
from sklearn.linear_model import Lasso
from sklearn.metrics import roc_auc_score
from sklearn.grid_search import GridSearchCV

lasso = GridSearchCV(Lasso(),
                    {'alpha': np.logspace(-10, -8, 5)})
lasso.fit(xtrain, ytrain)
lasso_preds = lasso.predict(xtest)
lasso_performance = (ytest, lasso_preds)
print 'Lasso: Area under the ROC curve = {}'.format(lasso_performance)

Lasso: Area under the ROC curve = (array([ 0.,  0.,  0., ...,  0.,  0.,  0.]), array([-0.01110595,  0.00548702,  0.02903895, ..., -0.00785013,
       -0.00217249, -0.00817287]))


In [21]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score
from sklearn.grid_search import GridSearchCV

gbm = GradientBoostingClassifier(n_estimators=500)

gbm.fit(xtrain, ytrain)
gbm_preds = gbm.predict_proba(xtest)[:, 1]
gbm_performance = roc_auc_score(ytest, gbm_preds)

print 'GBM: Area under the ROC curve ={}'.format(gbm_performance)

GBM: Area under the ROC curve =0.973048391666


In [22]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score

tree = GridSearchCV(DecisionTreeClassifier(),
                   {'max_depth': np.arange(3, 10)})
tree.fit(xtrain, ytrain)
tree_preds = tree.predict_proba(xtest)[:, 1]
tree_perf = roc_auc_score(ytest, tree_preds)

print "Decision Tree: Area under the ROC curve = {}".format(tree_perf)

Decision Tree: Area under the ROC curve = 0.9119433573


In [23]:
# See what variables the GBM model thinks are most useful

importances = pd.Series(gbm.feature_importances_, index=data.columns)
print importances.order(ascending=False)[:10]

STRATUM     0.083548
WEIGHT      0.063752
DRINKING    0.060818
EJECT_IM    0.060524
CASENUM     0.050680
HARM_EV     0.041842
AGE_IM      0.040943
IMPACT1     0.039241
ALC_RES     0.038808
HOSPITAL    0.032949
dtype: float64
